In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from datetime import datetime
import bs4
import pandas as pd
import time
driver = webdriver.Chrome()
driver.get('https://th.tripadvisor.com/Attraction_Review-g303915-d7923675-Reviews-Tha_Sadet_Market-Nong_Khai_Nong_Khai_Province.html')

review_data = pd.DataFrame(columns = ['name', 'city', 'score','title','comment', 'date'])

time.sleep(10.0)

def printtimestamp():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)

def getmonth(mtxt):
    if(mtxt == "มกราคม"):
        return "01"
    elif(mtxt == "กุมภาพันธ์"):
        return "02"
    elif(mtxt == "มีนาคม"):
        return "03"
    elif(mtxt == "เมษายน"):
        return "04"
    elif(mtxt == "พฤษภาคม"):
        return "05"
    elif(mtxt == "มิถุนายน"):
        return "06"
    elif(mtxt == "กรกฎาคม"):
        return "07"
    elif(mtxt == "สิงหาคม"):
        return "08"
    elif(mtxt == "กันยายน"):
        return "09"
    elif(mtxt == "ตุลาคม"):
        return "10"
    elif(mtxt == "พฤศจิกายน"):
        return "11"
    else:
        return "12"

def loaddata():
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    
    all_review_score = soup.find_all('svg',{'class':'RWYkj d H0'})
    all_review_comment = soup.find_all('span',{'class':'NejBf'})
    all_review_date = soup.find_all('div',{'class':'WlYyy diXIH cspKb bQCoY'})
    all_review_user = soup.find_all('div',{'class':'cjhIj'})

    all_review_score_list = []
    all_review_comment_list = []
    all_review_comment_header_list = []
    all_review_date_list = []
    all_review_user_list = []
    all_review_user_city_list = []

    for rev in all_review_user:    
        if(rev.span.a != None):
            all_review_user_list.append(rev.span.a.text)        
        if(rev.div != None and len(rev.div.text.strip()) != 0):
            all_review_user_city_list.append(rev.div.text.split()[0].replace(',','') if ',' in rev.div.text.split()[0] else rev.div.text.split()[0])
        else:
            all_review_user_city_list.append("")

    for review in all_review_date:
        if len(review.text) > 0:
            date = review.text.split()[1] if len(review.text.split()[1]) == 2 else "0" + review.text.split()[1]
            year = review.text.split()[4][2:4]
            month = getmonth(review.text.split()[2])
            datetime_txt = date + "/" + month + "/" + year
            #review_date = datetime.strptime(datetime_txt, '%d/%m/%y')
            all_review_date_list.append(datetime_txt)

    i = 1
    while i < len(all_review_score):
        review = all_review_score[i]
        if(len(review['aria-label'].strip()) != 0):
            print(review['aria-label'].strip())
            all_review_score_list.append(float(review['aria-label'].split()[1]))
        i += 1

    i = 0
    while i < len(all_review_comment) :    
        all_review_comment_header_list.append(all_review_comment[i].text)
        all_review_comment_list.append(all_review_comment[i+1].text)
        i+=2
    
    df = pd.DataFrame([all_review_user_list, all_review_user_city_list, all_review_score_list, all_review_comment_header_list, all_review_comment_list, all_review_date_list])
    df = df.transpose()
    df.columns = ['name', 'city', 'score', 'title', 'comment', 'date']
    df.dropna(subset = ['title'], inplace=True)
    return df

def addtodf(review_data, df):
    review_data = pd.concat([review_data, df], ignore_index=True)
    return review_data

df = loaddata()
review_data = addtodf(review_data, df)

In [ ]:
buttons_next = driver.find_elements(By.CLASS_NAME, 'cCnaz')
last_element = buttons_next[-1]
while last_element.is_displayed():
    ActionChains(driver).click(last_element).perform()  
    time.sleep(3.0)
    df = loaddata()
    review_data = addtodf(review_data, df)
    print(str(len(df)) + " to " + str(len(review_data)) + " at " + str(printtimestamp())) 
    buttons_next = driver.find_elements(By.CLASS_NAME, 'cCnaz')
    last_element = buttons_next[-1]

In [439]:
review_data.to_excel(r'results\review1.xlsx')